<a href="https://colab.research.google.com/github/urosgodnov/BigData/blob/master/textMining_scraping_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Permanently install libraries

## Initializing gdrive

In [1]:
# permanently install packageg in google colab
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Creating links

In [5]:
import os
nb_path = '/content/drive/MyDrive/Google_Colab_modules'

if not os.path.exists(nb_path):
  os.symlink('/content/drive/MyDrive/Google_Colab_modules', nb_path)  # Create the symlink only if it doesn't exist
else:
  print(f"Path '{nb_path}' already exists. Skipping symlink creation.")


Path '/content/drive/MyDrive/Google_Colab_modules' already exists. Skipping symlink creation.


## Installing to destination folder in gdrive

In [ ]:
#install the module in the
!pip install --target=$nb_path selenium

In [24]:
!pip install --target=$nb_path webdriver_manager

## Connecting the path

In the future, while all modules will be installed, we will only use this command.

In [25]:
import sys

sys.path.append("/content/drive/MyDrive/Google_Colab_modules")

# Webscraping

In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    ElementClickInterceptedException,
    TimeoutException
)
from datetime import datetime



In [ ]:
import pandas as pd
import time

In [44]:
options = webdriver.ChromeOptions()
options.add_argument("--verbose")
options.add_argument('--no-sandbox')
#options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument("--window-size=1920, 1200")
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=Service(),options=options)

In [30]:
from selenium.webdriver.chrome.options import Options
import google_colab_selenium as gs

custom_options = Options()


driver = gs.Chrome(options=custom_options)

<IPython.core.display.Javascript object>

In [49]:
l="https://www.imdb.com/title/tt1745960/reviews?ref_=tt_urv"
driver.get(l)

In [47]:
while True:
  try:
    driver.find_element("id","load-more-trigger").click()
    time.sleep(2)
  except:
    break

In [36]:
df = pd.DataFrame(columns=["Author","Title","Rating","Text","Date"])



In [37]:
from bs4 import BeautifulSoup

In [38]:
content = driver.page_source
soup = BeautifulSoup(content)

In [39]:

result=[]

In [50]:
def GetAReview(review):
  author = rev.findAll('a', attrs={'class':'ipc-link ipc-link--base'})[0].text
  title = rev.find('h3', attrs={'class':'ipc-title__text'}).text.strip()
  rating =rev.findAll('span', attrs={'class':'ipc-rating-star--rating'})[0].text.replace("\n","").strip()
  review = rev.findAll('div', attrs={'class':'ipc-html-content-inner-div'})[0].text
  date = rev.findAll('li', attrs={'class':'ipc-inline-list__item review-date'})[0].text
  date=datetime.strptime(date, '%b %d, %Y').strftime('%Y-%m-%d')

  return pd.DataFrame({"Author":author, "Title":title, "Rating":rating, "Text":review, "Date":date},index=[0])


In [41]:
for rev in soup.findAll('article'):
  try:
    result.append(GetAReview(rev))
  except:
    continue

In [42]:
df=pd.concat(result, axis=0, ignore_index=True)

ValueError: No objects to concatenate

In [43]:
df.head()

,Author,Title,Rating,Text,Date


In [ ]:
df.to_csv("Mavericks.csv")